In [4]:
from pydantic import BaseModel, Field
import json
from enum import Enum
from typing import Dict, List, Optional, Any, Union

def convert_to_gemini_schema(pydantic_schema: Dict[str, Any], root_schema: Dict[str, Any] = None) -> Dict[str, Any]:
    # Store the root schema for resolving references if not passed in
    if root_schema is None:
        root_schema = pydantic_schema
    
    gemini_schema = {}
    
    # Handle $ref references
    if "$ref" in pydantic_schema:
        ref_path = pydantic_schema["$ref"]
        # Handle different reference patterns
        if ref_path.startswith("#/"):
            # Split the path into components
            parts = ref_path.lstrip("#/").split("/")
            
            # Navigate through the schema to find the referenced object
            current = root_schema
            for part in parts:
                if part in current:
                    current = current[part]
                else:
                    # Reference not found
                    break
            
            # If we found the referenced schema, convert it
            if current != root_schema:
                referenced_schema = convert_to_gemini_schema(current, root_schema)
                return referenced_schema
    
    # Handle Optional types (which Pydantic often represents with anyOf)
    if "anyOf" in pydantic_schema:
        # Check if one of the options is "null" (which indicates Optional)
        null_present = any(schema.get("type") == "null" for schema in pydantic_schema["anyOf"])
        
        # If null is one of the options, it's an Optional type
        if null_present:
            # Find the non-null type and use it as the base
            for schema in pydantic_schema["anyOf"]:
                if schema.get("type") != "null":
                    # Save the description before processing the non-null schema
                    description = pydantic_schema.get("description")
                    title = pydantic_schema.get("title")
                    
                    # Use the non-null schema as our base
                    non_null_schema = convert_to_gemini_schema(schema, root_schema)
                    gemini_schema.update(non_null_schema)
                    gemini_schema["nullable"] = True
                    
                    # Restore the description and title if they exist
                    if description:
                        gemini_schema["description"] = description
                    if title and "description" not in gemini_schema:
                        gemini_schema["description"] = title
                    
                    return gemini_schema
    
    # Map basic type
    if "type" in pydantic_schema:
        # Convert Pydantic types to Gemini types
        type_mapping = {
            "string": "STRING",
            "number": "NUMBER",
            "integer": "INTEGER",
            "boolean": "BOOLEAN",
            "array": "ARRAY",
            "object": "OBJECT"
        }
        gemini_schema["type"] = type_mapping.get(pydantic_schema["type"], pydantic_schema["type"])
    
    # Map title and description
    if "description" in pydantic_schema:
        gemini_schema["description"] = pydantic_schema["description"]
    elif "title" in pydantic_schema:
        gemini_schema["description"] = pydantic_schema["title"]
    
    # Map format if available
    if "format" in pydantic_schema:
        gemini_schema["format"] = pydantic_schema["format"]
    
    # Map enum values
    if "enum" in pydantic_schema:
        gemini_schema["enum"] = pydantic_schema["enum"]
        if "type" not in gemini_schema and len(pydantic_schema["enum"]) > 0:
            # If type is missing but we have enum values, assume STRING
            gemini_schema["type"] = "STRING"
            gemini_schema["format"] = "enum"
    
    # Map array constraints
    if "maxItems" in pydantic_schema:
        gemini_schema["maxItems"] = str(pydantic_schema["maxItems"])
    if "minItems" in pydantic_schema:
        gemini_schema["minItems"] = str(pydantic_schema["minItems"])
    
    # Map properties
    if "properties" in pydantic_schema:
        gemini_schema["properties"] = {}
        for prop_name, prop_schema in pydantic_schema["properties"].items():
            gemini_schema["properties"][prop_name] = convert_to_gemini_schema(prop_schema, root_schema)
    
    # Map required fields
    if "required" in pydantic_schema:
        gemini_schema["required"] = pydantic_schema["required"]
    
    # Map items for arrays
    if "items" in pydantic_schema:
        gemini_schema["items"] = convert_to_gemini_schema(pydantic_schema["items"], root_schema)
    
    # Add propertyOrdering if needed
    if "properties" in pydantic_schema:
        gemini_schema["propertyOrdering"] = list(pydantic_schema["properties"].keys())
    
    return gemini_schema

# Test with a schema using $defs instead of definitions
schema_with_defs = {
    "title": "MainModel",
    "description": "This is the main model with $defs",
    "type": "object",
    "properties": {
        "user": {"$ref": "#/$defs/User"},
        "status": {"$ref": "#/$defs/Status"}
    },
    "required": ["user"],
    "$defs": {
        "User": {
            "title": "User",
            "type": "object",
            "properties": {
                "id": {"title": "ID", "type": "integer"},
                "name": {"title": "Name", "type": "string"}
            },
            "required": ["id", "name"]
        },
        "Status": {
            "title": "Status",
            "enum": ["active", "inactive", "pending"],
            "type": "string"
        }
    }
}

# Convert to Gemini schema format
gemini_schema = convert_to_gemini_schema(schema_with_defs)

# Print the result
print(json.dumps(gemini_schema, indent=2))